In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json

In [2]:
from glob import glob

files = glob('url/*.json')
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        urls.extend(json.load(fopen))
urls = sorted(list(set(urls)))
len(urls), len(files)

100%|███████████████████████████████████| 49312/49312 [00:07<00:00, 6661.63it/s]


(1479312, 49312)

In [3]:
urls[0]

'https://malaysiastock.biz/Company-Announcement.aspx?id=100000'

In [4]:
# r = requests.get(urls[0])
# soup = BeautifulSoup(r.content, "lxml")

In [5]:
# soup.find('h1')

In [6]:
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'


def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()

def parse_tika(string, minlen=1,):
    while True:
        try:
            raw_xml = parser.from_buffer(string, TIKA_HOST, xmlContent=True)
            break
        except BaseException:
            time.sleep(1.0)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            if len(t_) > minlen:
                t.append(t_)
        except Exception as e:
            pass

    return t

In [7]:
# parse_tika(str(soup.find('div', {'id': 'main'})))

In [8]:
def crawl(url, filename):
    if os.path.exists(filename):
        return [True]
    
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)

    soup = BeautifulSoup(r.content, "lxml")

    body = parse_tika(str(soup.find('div', {'id': 'main'})))
    
    data = {
        'url': url,
        'body': body,
    }
    with open(filename, 'w') as fopen:
        json.dump(data, fopen)

In [9]:
# !rm -rf page
!mkdir page

mkdir: cannot create directory ‘page’: File exists


In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

for i in tqdm(range(0, len(urls), max_worker)):
    b = urls[i: i + max_worker]
    urls_ = [(b[k], os.path.join('page', f'{i + k}.json')) for k in range(len(b))]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(crawl, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 71%|██████████████████████▏        | 105674/147932 [5:03:24<4:18:45,  2.72it/s]